### 1. Importar Lista de vehículos ligeros eléctricos con precios en Europa "WEB SCRAPING":

In [143]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [187]:
# URL base de la página a hacer scraping
base_url = 'https://ev-database.org/'
url = base_url

# Lista para almacenar los datos
data = []

# Realizar solicitudes GET a cada página
while True:
    # Realizar la solicitud GET a la página
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Parsear el contenido HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos los elementos <div> con la clase 'list-item'
        list_items = soup.find_all('div', class_='list-item')

        # Iterar sobre los elementos 'list-item' y extraer información
        for item in list_items:
            # Buscar el elemento <div> con la clase 'item-data'
            item_data = item.find('div', class_='item-data')
            if item_data:
                # Encontrar el nombre del vehículo
                car_name = item_data.find('a', class_='title').text.strip()

                # Encontrar la velocidad máxima
                top_speed_tag = item_data.find('span', class_='topspeed')
                top_speed = top_speed_tag.text.strip() if top_speed_tag else "Velocidad máxima no encontrada"

                # Encontrar el tiempo de 0 a 100 km/h
                acceleration_tag = item_data.find('span', class_='acceleration')
                acceleration = acceleration_tag.text.strip() if acceleration_tag else "Tiempo de 0 a 100 km/h no encontrado"

                # Encontrar el rango
                range_tag = item_data.find('span', class_='erange_real')
                range_value = range_tag.text.strip() if range_tag else "Rango no encontrado"

                # Encontrar la velocidad de carga rápida
                fastcharge_speed_tag = item_data.find('span', class_='fastcharge_speed_print')
                fastcharge_speed = fastcharge_speed_tag.text.strip() if fastcharge_speed_tag else "Velocidad de carga rápida no encontrada"

                # Encontrar el contenedor de precios
                pricing_container = item.find('div', class_='pricing')
                if pricing_container:
                    # Encontrar todos los elementos de precio
                    pricing_elements = pricing_container.find_all('div', class_='price_buy')
                    
                    # Crear un diccionario para almacenar los precios de cada país
                    prices = {}
                    
                    # Iterar sobre los elementos de precio
                    for pricing_element in pricing_elements:
                        # Extraer el precio y el país
                        price_text = pricing_element.find('span').text.strip()
                        country_code = pricing_element.find('span', class_='flag-icon').attrs['class'][1].replace('country_', '').upper()
                        # Agregar el precio al diccionario de precios
                        prices[country_code] = price_text
                
                    # Agregar los datos a la lista
                    data.append({
                        "Nombre del vehículo": car_name,
                        "Velocidad máxima": top_speed,
                        "Tiempo de 0 a 100 km/h": acceleration,
                        "Rango": range_value,
                        "Velocidad de carga rápida": fastcharge_speed,
                        "Precios por país": prices
                    })

        # Encontrar el enlace a la siguiente página, si existe
        next_page_link = soup.find('a', class_='next')
        if next_page_link:
            url = base_url + next_page_link['href']
        else:
            break  # Salir del bucle si no hay más páginas disponibles
    else:
        print('Error al acceder a la página:', response.status_code)
        break


In [188]:
data

[{'Nombre del vehículo': 'Tesla Model 3',
  'Velocidad máxima': '201 km/h',
  'Tiempo de 0 a 100 km/h': '6.1 sec',
  'Rango': '415 km',
  'Velocidad de carga rápida': '690 km/h',
  'Precios por país': {'FLAG-ICON-DE': '€43,970',
   'FLAG-ICON-NL': '€43,993',
   'FLAG-ICON-GB': '£39,990'}},
 {'Nombre del vehículo': 'Tesla Model Y Long Range Dual Motor',
  'Velocidad máxima': '217 km/h',
  'Tiempo de 0 a 100 km/h': '5.0 sec',
  'Rango': '435 km',
  'Velocidad de carga rápida': '670 km/h',
  'Precios por país': {'FLAG-ICON-DE': '€55,970',
   'FLAG-ICON-NL': '€50,990',
   'FLAG-ICON-GB': '£52,990'}},
 {'Nombre del vehículo': 'Tesla Model Y',
  'Velocidad máxima': '217 km/h',
  'Tiempo de 0 a 100 km/h': '6.9 sec',
  'Rango': '350 km',
  'Velocidad de carga rápida': '580 km/h',
  'Precios por país': {'FLAG-ICON-DE': '€45,970',
   'FLAG-ICON-NL': '€43,990',
   'FLAG-ICON-GB': '£44,990'}},
 {'Nombre del vehículo': 'BYD ATTO 3',
  'Velocidad máxima': '160 km/h',
  'Tiempo de 0 a 100 km/h': '7.3

### 2. Corregir formato de números y renombrar columnas:

In [189]:
# Crear un DataFrame de Pandas con los datos
df = pd.DataFrame(data)

# Renombrar las columnas
df.rename(columns={
    'Velocidad máxima': 'Velocidad Máxima (Km/h)',
    'Tiempo de 0 a 100 km/h': 'Aceleración 0 a 100 Km/h (seg)',
    'Rango': 'Rango (Km)',
    'Velocidad de carga rápida': 'Carga rápida (Km/h)'
}, inplace=True)

# Limpiar los datos numéricos
df['Velocidad Máxima (Km/h)'] = df['Velocidad Máxima (Km/h)'].str.extract('(\d+)').astype(float)
df['Aceleración 0 a 100 Km/h (seg)'] = df['Aceleración 0 a 100 Km/h (seg)'].str.extract('(\d+\.?\d*)').astype(float)
df['Rango (Km)'] = df['Rango (Km)'].str.extract('(\d+)').astype(float)
df['Carga rápida (Km/h)'] = df['Carga rápida (Km/h)'].str.extract('(\d+)').astype(float)


### 3. Renombrar columnas de precios y arreglar valores:

In [190]:
# Suponiendo que df es tu DataFrame y 'precios por país' es la columna con los precios
# Crea columnas vacías para los precios de cada país
df['Precio Alemania (€)'] = ''
df['Precio Holanda (€)'] = ''
df['Precio Inglaterra (£)'] = ''

# Itera sobre las filas del DataFrame
for index, row in df.iterrows():
    # Extrae el diccionario de precios de la columna 'precios por país'
    precios_dict = row['Precios por país']
    
    # Extrae el precio para Alemania
    if 'FLAG-ICON-DE' in precios_dict:
        df.at[index, 'Precio Alemania (€)'] = precios_dict['FLAG-ICON-DE']
    
    # Extrae el precio para Holanda
    if 'FLAG-ICON-NL' in precios_dict:
        df.at[index, 'Precio Holanda (€)'] = precios_dict['FLAG-ICON-NL']
    
    # Extrae el precio para Inglaterra
    if 'FLAG-ICON-GB' in precios_dict:
        df.at[index, 'Precio Inglaterra (£)'] = precios_dict['FLAG-ICON-GB']



In [191]:
# Quitar el símbolo del euro (€) de las columnas de precios de Alemania y Holanda
df['Precio Alemania (€)'] = df['Precio Alemania (€)'].str.replace('€', '')
df['Precio Holanda (€)'] = df['Precio Holanda (€)'].str.replace('€', '')

# Quitar el símbolo de la libra esterlina (£) de la columna de precios de Inglaterra
df['Precio Inglaterra (£)'] = df['Precio Inglaterra (£)'].str.replace('£', '')

# Reemplazar "N/A" con NaN
df['Precio Alemania (€)'] = df['Precio Alemania (€)'].replace('N/A', np.nan)
df['Precio Holanda (€)'] = df['Precio Holanda (€)'].replace('N/A', np.nan)
df['Precio Inglaterra (£)'] = df['Precio Inglaterra (£)'].replace('N/A', np.nan)


# Quitar las comas y asteriscos de los valores y convertirlos a tipo numérico
df['Precio Alemania (€)'] = df['Precio Alemania (€)'].str.replace(',', '').str.replace('*', '').astype(float)
df['Precio Holanda (€)'] = df['Precio Holanda (€)'].str.replace(',', '').str.replace('*', '').astype(float)
df['Precio Inglaterra (£)'] = df['Precio Inglaterra (£)'].str.replace(',', '').str.replace('*', '').astype(float)

### 4. Unificar precios a promedio Europa en Euros:

In [192]:
# Crear la columna "Europa (€)" que promedia los valores de las tres columnas de precios, ignorando los valores nulos
df['Europa (€)'] = df[['Precio Alemania (€)', 'Precio Holanda (€)', 'Precio Inglaterra (£)']].mean(axis=1, skipna=True)

# Eliminar las columnas individuales de precios
df.drop(columns=['Precio Alemania (€)', 'Precio Holanda (€)', 'Precio Inglaterra (£)','Precios por país'], inplace=True)


### 5. Eliminar registros nulos y duplicados:

In [193]:
# Eliminar registros nulos
df.dropna(inplace=True)

# Eliminar registros duplicados
df.drop_duplicates(inplace=True)

df['Europa (€)']=df['Europa (€)'].round(decimals=2)

Los datos se dejaron bajar una sola vez y se bloqueó la IP, se requiere un plan pago de la api para acceder a los datos en esta ubicación: data-services@ev-database.org; solicitando el uso de datos pago en esta url: data-services@ev-database.org

### 6. Cambio de precio a Dolar:

revisar si GCP soporta la librería forex_python

In [194]:
# Convertir a dolar mediante búsqueda de la tasa Scraping a "https://www.google.com/finance/quote/EUR-USD?sa=X&ved=2ahUKEwjUhIO6x9SFAxUjfDABHXpWBXEQmY0JegQIGhAv"

# URL de la página con la tasa de cambio
url = "https://www.google.com/finance/quote/EUR-USD?sa=X&ved=2ahUKEwjUhIO6x9SFAxUjfDABHXpWBXEQmY0JegQIGhAv"

# Realizar la solicitud GET a la página
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")
    
# Encontrar el elemento con la clase "YMlKec fxKbKc"
div_tasa_cambio = soup.find("div", class_="YMlKec fxKbKc")

tasa_cambio = div_tasa_cambio.text.strip().replace(',', '.')  # Reemplazar "," por "." para convertir a número decimal
tasa_cambio = float(div_tasa_cambio.text.strip().replace(',', '.'))

# Obtener la tasa de conversión actual de EUR a USD
tasa_conversion = tasa_cambio

df['Precio($USD)']=round(df['Europa (€)']*tasa_conversion,2)

df=df.drop(columns=['Europa (€)'])


In [195]:
df

,Nombre del vehículo,Velocidad Máxima (Km/h),Aceleración 0 a 100 Km/h (seg),Rango (Km),Carga rápida (Km/h),Precio($USD)
0,Tesla Model 3,201.0,6.1,415.0,690.0,45461.70
1,Tesla Model Y Long Range Dual Motor,217.0,5.0,435.0,670.0,56830.24
2,Tesla Model Y,217.0,6.9,350.0,580.0,47947.73
3,BYD ATTO 3,160.0,7.3,330.0,370.0,40566.37
4,Tesla Model Y Long Range RWD,217.0,5.9,460.0,710.0,53263.02
...,...,...,...,...,...,...
344,Porsche Taycan Turbo Cross Turismo,250.0,2.8,495.0,1090.0,176477.51
345,Mercedes-Benz eVito Tourer Extra-Long 60 kWh,160.0,12.0,215.0,200.0,73876.46
346,Mercedes-Benz eVito Tourer Long 60 kWh,160.0,12.0,220.0,210.0,72905.43
347,Porsche Taycan Turbo Sport Turismo,260.0,2.7,470.0,1030.0,176015.62
